In [10]:
import psycopg2
import json
import codecs
from ast import literal_eval
import re

<h1> connect to db and create table </h1>

In [3]:
conn = psycopg2.connect("host=localhost port=5432 dbname=amazon user=postgres password=darkmatter")
cur = conn.cursor()

<h1> Create a Table with json as the column </h1> 

In [4]:
#QA db
cur.execute("CREATE TABLE qa (id serial PRIMARY KEY,\
    catagory text,\
    questionType text,\
    answerType text,\
    asin text,\
    answerTime TimeStamp,\
    unixTime int,\
    question text,\
    answer text\
    );")
conn.commit() #submit change to db

<h1> function to write values to SQL </h1>

In [5]:
def writeJSON2SQL(catagory,questionType,answerType,asin,answerTime,unixTime,question,answer):
    if answerTime=='':
        answerTime=None
    try:
        unixTime=int(unixTime)
    except ValueError:
        unixTime=0
    
    cur.execute("INSERT INTO qa (\
    catagory,\
    questionType,\
    answerType,\
    asin,\
    answerTime,\
    unixTime,\
    question,\
    answer\
    )\
    VALUES (\
    %s,%s,%s,%s,%s,%s,%s,%s\
    )",(catagory,questionType,answerType,asin,answerTime,unixTime,question,answer)\
    )
    conn.commit() #submit change to db

<h1> read the data </h1>

In [6]:
def read_line(line):
    questionType=re.findall(r'questionType\': \'(.*?)\',',line)
    if questionType==[]:
        questionType=['']
    answerType=re.findall(r'answerType\': \'(.*?)\',',line)
    if answerType==[]:
        answerType=['']
    asin=re.findall(r'asin\': \'(.*?)\',',line)
    answerTime=re.findall(r'answerTime\': \'(.*?)\',',line)
    if answerTime==[]:
        answerTime=['']
    if answerTime[0].count(" ago")>0:
        answerTime=['']
    unixTime=re.findall(r'unixTime\': (.*?),',line)
    if unixTime==[]:
        unixTime=['']
    question=re.findall(r'question\': \'(.*?)\',',line)
    if question==[]:
        question=re.findall(r'question\': "(.*?)"',line)
    answer=re.findall(r'answer\': \'(.*?)\'',line)
    if answer==[]:
        answer=re.findall(r'answer\': "(.*?)"',line)
    return questionType[0],answerType[0],asin[0],answerTime[0],unixTime[0],question[0],answer[0]

<h1> read in files and populate sql db </h1>

In [ ]:
file_list=!ls /home/ubuntu/amazon/QA
for catagory_file in file_list:
    lines=open('/home/ubuntu/amazon/QA/'+catagory_file).read().split('\n')[:-1]
    for ii,line in enumerate(lines):
        questionType,answerType,asin,answerTime,unixTime,question,answer = read_line(line)
        writeJSON2SQL(catagory_file,questionType,answerType,asin,answerTime,unixTime,question,answer)